In [1]:
import pandas as pd

import sys

sys.path.append('.')
from tqdm.notebook import tqdm
from copy import deepcopy


from time_forecasting import *
from fishs_config import *

In [2]:
dt = pd.read_csv(r'..\day_forecasts0.csv', sep=';')
dt

,day_temp,day_pressure,day_obl,day_phen,day_dir,day_wind,areal,city,year,month,...,Красноперка,Налим,Густера,Амур,Ерш,Сазан,Подуст,Толстолобик,Вобла,Хариус
0,-4,749,dull,snow,Ю,3,Алтайский край,Барнаул,2020,1,...,0.256061,0.577778,0.588095,0.0,0.282609,0.040625,0.0,0.0,0.185714,0.094223
1,-1,750,dull,-,Ю,5,Алтайский край,Барнаул,2020,1,...,0.151010,0.317778,0.371429,0.0,0.406250,0.028676,0.0,0.0,0.140693,0.077836
2,-3,749,dull,-,Ю,2,Алтайский край,Барнаул,2020,1,...,0.216667,0.577778,0.464286,0.0,0.441576,0.040625,0.0,0.0,0.225108,0.110609
3,-6,753,dull,-,Ю,1,Алтайский край,Барнаул,2020,1,...,0.177273,0.577778,0.464286,0.0,0.264946,0.032261,0.0,0.0,0.225108,0.110609
4,-2,752,suncl,-,Ю,3,Алтайский край,Барнаул,2020,1,...,0.242929,0.606667,0.402381,0.0,0.635870,0.052574,0.0,0.0,0.270130,0.147479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30516,-6,752,dull,-,З,1,Московская область,Щелково,2020,12,...,0.180556,0.252778,0.417857,0.0,0.317935,0.000000,0.0,0.0,0.225108,0.110609
30517,-6,759,sunc,-,ЮВ,1,Московская область,Щелково,2020,12,...,0.235269,0.265417,0.473571,0.0,0.360326,0.000000,0.0,0.0,0.270130,0.147479
30518,-5,754,dull,-,ЮВ,3,Московская область,Щелково,2020,12,...,0.054714,0.113750,0.083571,0.0,0.678261,0.000000,0.0,0.0,0.123810,0.049160
30519,-3,753,dull,-,-,0,Московская область,Щелково,2020,12,...,0.180556,0.353889,0.306429,0.0,0.720652,0.000000,0.0,0.0,0.225108,0.126996


In [3]:
i = 100
dt[i:i + 10][['Щука', 'Карп', 'Налим', 'Амур', 'Окунь']].plot()

In [4]:
gen = DayGenerator(dt)

In [11]:
init_ = False
train_dt = pd.DataFrame()
for i, row in tqdm(dt.iterrows()):
    if i < 1 or i > len(dt) - 2:
        continue
#     if i > 20000:
#         break
    day_rows = []
    day = gen[i]
    for day_id in range(8):
        temp_dict = {'time': times[day_id]}
        for key in day:
            temp = day[key].split(',') if isinstance(day[key], str) else day[key]
            temp_dict.update({key: temp[day_id] if isinstance(day[key], str) else temp})
        for fish in fishs:
            fish = fish.capitalize()
#             fish_dict = deepcopy(temp_dict)
            temp_dict[fish] = gen_forecast(row[fish], times[day_id], fish)
#             fish_dict['forecast'] = gen_forecast(row[fish], times[day_id], fish)
        day_rows.append(temp_dict)
    if not init_:
        train_dt = pd.DataFrame(day_rows)
        init_ = True
        continue
    train_dt = pd.concat([train_dt, pd.DataFrame(day_rows)], ignore_index=True)


In [12]:
train_dt

,time,pressure,temperature,wind,gust,wind_direction,humidity,phenomenon,uv_index,moon_direction,...,Красноперка,Налим,Густера,Амур,Ерш,Сазан,Подуст,Толстолобик,Вобла,Хариус
0,0,750,-10,1,5,ЮЗ,50,пасмурно,0,-1,...,0.105707,0.476667,0.260000,0.0,0.284375,0.020074,0.0,0.0,0.098485,0.054485
1,3,750,-10,2,9,Ю,49,пасмурно,0,-1,...,0.105707,0.476667,0.260000,0.0,0.284375,0.020074,0.0,0.0,0.098485,0.054485
2,6,750,-7,3,9,Ю,51,пасмурно,0,-1,...,0.196313,0.349556,0.482857,0.0,0.528125,0.037279,0.0,0.0,0.182900,0.101187
3,9,750,-6,4,11,ЮЗ,45,пасмурно,0,-1,...,0.181212,0.222444,0.445714,0.0,0.487500,0.034412,0.0,0.0,0.168831,0.093403
4,12,750,-1,5,13,Ю,37,пасмурно,2,-1,...,0.166111,0.222444,0.408571,0.0,0.446875,0.031544,0.0,0.0,0.154762,0.085620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244147,9,752,-5,0,6,С,21,пасмурно,0,1,...,0.216667,0.247722,0.367714,0.0,0.864783,0.000000,0.0,0.0,0.270130,0.152395
244148,12,753,-3,0,5,СВ,21,пасмурно,2,1,...,0.198611,0.247722,0.337071,0.0,0.792717,0.000000,0.0,0.0,0.247619,0.139695
244149,15,753,-3,0,7,С,24,пасмурно,1,1,...,0.180556,0.247722,0.306429,0.0,0.720652,0.000000,0.0,0.0,0.225108,0.126996
244150,18,751,-3,0,3,С,27,облачно,0,1,...,0.234722,0.247722,0.398357,0.0,0.936848,0.000000,0.0,0.0,0.292641,0.165095


In [13]:
train_dt.to_csv(r'..\time_multiple_forecasts0.csv', sep=';', index=False)